In [2]:
from typing import Annotated, Literal

from pydantic import BaseModel, Discriminator, Field, Tag


class Cat(BaseModel):
    pet_type: Literal['cat']
    age: int


class Dog(BaseModel):
    pet_kind: Literal['dog']
    age: int


def pet_discriminator(v):
    if isinstance(v, dict):
        return v.get('pet_type', v.get('pet_kind'))
    return getattr(v, 'pet_type', getattr(v, 'pet_kind', None))


class Model(BaseModel):
    pet: Annotated[Cat, Tag('cat')] | Annotated[Dog, Tag('dog')] = Field(
        discriminator=Discriminator(pet_discriminator)
    )


print(repr(Model.model_validate({'pet': {'pet_type': 'cat', 'age': 12}})))
#> Model(pet=Cat(pet_type='cat', age=12))

print(repr(Model.model_validate({'pet': {'pet_kind': 'dog', 'age': 12}})))

Model.model_json_schema()

#> Model(pet=Dog(pet_kind='dog', age=12))

Model(pet=Cat(pet_type='cat', age=12))
Model(pet=Dog(pet_kind='dog', age=12))


{'$defs': {'Cat': {'properties': {'pet_type': {'const': 'cat',
     'title': 'Pet Type',
     'type': 'string'},
    'age': {'title': 'Age', 'type': 'integer'}},
   'required': ['pet_type', 'age'],
   'title': 'Cat',
   'type': 'object'},
  'Dog': {'properties': {'pet_kind': {'const': 'dog',
     'title': 'Pet Kind',
     'type': 'string'},
    'age': {'title': 'Age', 'type': 'integer'}},
   'required': ['pet_kind', 'age'],
   'title': 'Dog',
   'type': 'object'}},
 'properties': {'pet': {'oneOf': [{'$ref': '#/$defs/Cat'},
    {'$ref': '#/$defs/Dog'}],
   'title': 'Pet'}},
 'required': ['pet'],
 'title': 'Model',
 'type': 'object'}